In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc, count
spark = SparkSession.builder.appName('trial_mini_project').getOrCreate()



# importing data from datastet

df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
#df_q2 = df.select('Dest', 'TailNum').where(df["Dest"] != '0')



In [28]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- C

# 1) Find the most frequent tail number which is getting in destination 


In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc,expr

#importing data from local data folder
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

#selecting the required columns , this makes processing faster
df_q2 = df.select('Dest','TailNum')

df_q2_no_null = df_q2.filter((df_q2["TailNum"] != "0") & (df_q2["TailNum"] != "000000")) #removing rows with TailNum = 0/000000

print(type(df_q2))
print(" +++++ "*20)
most_frequent = df_q2_no_null.groupBy("TailNum",'Dest').agg(count("TailNum").alias("Count_TailNum")).sort(col("Count_TailNum").desc()).show()

#most_frequent.distinct().show()

<class 'pyspark.sql.dataframe.DataFrame'>
 +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++  +++++ 
+-------+----+-------------+
|TailNum|Dest|Count_TailNum|
+-------+----+-------------+
| N655BR| HNL|         2241|
| N651BR| HNL|         2173|
| N654BR| HNL|         2138|
| N693BR| HNL|         2067|
| N479HA| HNL|         2038|
| N478HA| HNL|         2024|
| N485HA| HNL|         1984|
| N480HA| HNL|         1976|
| N484HA| HNL|         1944|
| N487HA| HNL|         1909|
| N481HA| HNL|         1868|
| N810AL| HNL|         1843|
| N477HA| HNL|         1837|
| N837AL| HNL|         1836|
| N475HA| HNL|         1816|
| N486HA| HNL|         1787|
| N646BR| HNL|         1768|
| N836AL| HNL|         1761|
| N824AL| HNL|         1754|
| N808AL| HNL|         1740|
+-------+----+-------------+
only showing top 20 rows



# 2) Find out the cancelled flight  details for the last quarter of the year 2007



In [55]:
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df.na.drop()
df.filter((df.Month >= 8) & (df.Month <=12) & (df.Cancelled == 1)).show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2007|    8|         2|        4|     NA|       830|     NA|       930|           WN|        7

# 3) Find out the average weather delays for a particular flight per month


In [57]:
from pyspark.sql.functions import avg, col, desc

df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)

df.groupBy("Month").avg('WeatherDelay').sort(col("Month")).collect()

[Row(Month=1, avg(WeatherDelay)=0.8126742594025668),
 Row(Month=2, avg(WeatherDelay)=1.1426651862433788),
 Row(Month=3, avg(WeatherDelay)=0.6333765638468795),
 Row(Month=4, avg(WeatherDelay)=0.51643216930666),
 Row(Month=5, avg(WeatherDelay)=0.6052272846017077),
 Row(Month=6, avg(WeatherDelay)=1.2763936562420544),
 Row(Month=7, avg(WeatherDelay)=1.0766004687307265),
 Row(Month=8, avg(WeatherDelay)=0.8375915956275956),
 Row(Month=9, avg(WeatherDelay)=0.41135346150449775),
 Row(Month=10, avg(WeatherDelay)=0.45674389516057345),
 Row(Month=11, avg(WeatherDelay)=0.3357768086867862),
 Row(Month=12, avg(WeatherDelay)=1.1352771929481762)]

# 4) Inspite of NASDelay, SecurityDelay, LateAircraftDelay,Weatherdealy which flight reached on time



In [14]:
df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)


#NASDelay, SecurityDelay, LateAircraftDelay, Weatherdealy, ArrDelay
#df.filter((df.Month >= 8) & (df.Month <=12) & (df.Cancelled == 1)).show()
df.filter(((df.NASDelay > 0) | (df.SecurityDelay > 0) | (df.LateAircraftDelay > 0) | (df.WeatherDelay > 0)) & (df.ArrDelay <= 0) ).count()

0

# 5) Month wise total distance travelled by each flight number in every month

In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_q2 = df.select('Month','TailNum', 'Distance')

print(type(df_q2))

total_distance = df_q2.groupBy("Month").sum("Distance").sort(col("Month")).show()

#total_count = df_q2.groupBy("TailNum").sum("Distance").sort(col("sum(Distance)").desc()).show()





<class 'pyspark.sql.dataframe.DataFrame'>
+-----+-------------+
|Month|sum(Distance)|
+-----+-------------+
|    1|    441687538|
|    2|    402943192|
|    3|    458766108|
|    4|    441268345|
|    5|    453440873|
|    6|    456907218|
|    7|    474040538|
|    8|    473858891|
|    9|    431198160|
|   10|    450148203|
|   11|    433902481|
|   12|    446705757|
+-----+-------------+



# 6) Month wise how many flights get diverted(origin to destination)



In [74]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_q2 = df.select('Month','Diverted')


total_distance = df_q2.groupBy("Month").sum("Diverted").sort(col("Month")).show()

+-----+-------------+
|Month|sum(Diverted)|
+-----+-------------+
|    1|         1200|
|    2|         1261|
|    3|         1275|
|    4|         1193|
|    5|         1442|
|    6|         2199|
|    7|         2150|
|    8|         2101|
|    9|          962|
|   10|         1000|
|   11|          881|
|   12|         1515|
+-----+-------------+



# 7) Week and month wise number of trips in all the flights



In [105]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','DayOfWeek','TailNum')



total_distance = df_q2.groupBy("Month",'DayOfWeek','TailNum').agg(count("TailNum").alias("total_trips")).sort(['Month','DayOfWeek']).show()

+-----+---------+-------+-----------+
|Month|DayOfWeek|TailNum|total_trips|
+-----+---------+-------+-----------+
|    1|        1| N938UA|         21|
|    1|        1|   N793|         17|
|    1|        1| N27506|         13|
|    1|        1| N444YV|         30|
|    1|        1| N611SW|         39|
|    1|        1| N594SW|         17|
|    1|        1| N915EV|         24|
|    1|        1| N509AA|         21|
|    1|        1| N489UA|         14|
|    1|        1| N679DA|         20|
|    1|        1|  N6701|         11|
|    1|        1| N919DL|         20|
|    1|        1| N907EV|         23|
|    1|        1| N981AT|         25|
|    1|        1| N583NW|          9|
|    1|        1| N304US|         20|
|    1|        1| N372NW|         19|
|    1|        1| N601NW|         25|
|    1|        1| N5EFAA|         15|
|    1|        1| N4YBAA|         16|
+-----+---------+-------+-----------+
only showing top 20 rows



In [95]:
#total_distance.sort(['Month','DayOfWeek']).show()

# 8) Which flights covered maximum origin and destination by month wise


# 9) Average month wise arrival delay (flightnum wise)



In [102]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','ArrDelay','FlightNum')



total_distance = df_q2.groupBy("Month", "FlightNum").agg(avg("ArrDelay").alias("avg_monthly_arrival_delay")).sort(['Month']).show()

+-----+---------+-------------------------+
|Month|FlightNum|avg_monthly_arrival_delay|
+-----+---------+-------------------------+
|    1|       59|        6.102941176470588|
|    1|      239|        6.021739130434782|
|    1|     1972|        6.168539325842697|
|    1|       96|       10.427631578947368|
|    1|     2543|        5.857142857142857|
|    1|     1670|        6.069767441860465|
|    1|     1170|        19.62439024390244|
|    1|     1839|       2.7282608695652173|
|    1|     1703|         9.09727626459144|
|    1|      318|        8.606666666666667|
|    1|     2592|        1.911764705882353|
|    1|     1185|        12.43661971830986|
|    1|     1430|       15.057692307692308|
|    1|     1799|        6.566666666666666|
|    1|     2759|        42.69565217391305|
|    1|     3058|       10.226666666666667|
|    1|     3014|       1.8641975308641976|
|    1|     2898|       0.7391304347826086|
|    1|     7109|       16.894736842105264|
|    1|     5448|       1.425925

# 10) Average month wise departure delay (flightnum wise)

In [104]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, desc


df = spark.read.csv('Data/2007.csv',inferSchema=True,header=True)
df_no_cancel = df.filter('Cancelled== 0') 
df_q2 = df_no_cancel.select('Month','DepDelay','FlightNum')



total_distance = df_q2.groupBy("Month","FlightNum").agg(avg("DepDelay").alias("avg_monthly_departure_delay")).sort(['Month']).show()

+-----+---------+---------------------------+
|Month|FlightNum|avg_monthly_departure_delay|
+-----+---------+---------------------------+
|    1|     2992|          7.778947368421052|
|    1|     1167|           5.47093023255814|
|    1|     2004|          6.402439024390244|
|    1|     2099|         3.4107142857142856|
|    1|     1912|         11.926470588235293|
|    1|      864|          5.694117647058824|
|    1|      838|         10.076086956521738|
|    1|     1406|         10.136842105263158|
|    1|     2994|                       9.95|
|    1|     2843|          7.131147540983607|
|    1|     2668|          6.383720930232558|
|    1|     1192|                        8.0|
|    1|     3021|                   -4.28125|
|    1|     3030|          7.379310344827586|
|    1|     3169|          80.33333333333333|
|    1|     7157|          49.30555555555556|
|    1|     7047|         37.027027027027025|
|    1|     5025|          8.346153846153847|
|    1|     5379|          5.63333